Author: Joshua
Summary: Run a single instance of the two stage stochastic program
Input:
Output:

In [1]:
using Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

joshpath = "C:/Users/Owner/Documents/Austin/AmbulanceDeployment/AmbulanceDeployment.jl-legacy"

PROJECT_ROOT = joshpath
currentpath = joshpath

cd(currentpath)
Pkg.activate(".")

#include(".//src//problem.jl")
#include(".//src//model.jl")

using AmbulanceDeployment 
using DataFrames, Winston, JLD, CSV, Gurobi, JuMP, GLPK, Dates

   Updating registry at `C:\Users\Owner\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`
 Activating environment at `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
┌ Info: Precompiling AmbulanceDeployment [d2c08a37-75f3-4934-bcbc-64feba624d07]
└ @ Base loading.jl:1278
┌ Warning: Package AmbulanceDeployment does not have Random in its dependencies:
│ - If you have AmbulanceDeployment checked out for development and have
│   added Random as a dependency but hav

In [2]:
# hourly calls - regions x hours x number of calls/ per region per hour (e.g. 210 x 10000 x Z)
#adjacent_nbhd - boolean matrix, regionxregion (210x210) - if neighborhoods/region are adjacent?
#coverage - boolean matrix, regions x stations (210x44) - true if station is within coverage of region
#incidents - god object
hourly_calls = CSV.File(PROJECT_ROOT * "/test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
# weekend_hourly_calls = CSV.File("data/processed/2-weekend_calls.csv") |> DataFrame
adjacent_nbhd = CSV.File(PROJECT_ROOT * "/test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read(PROJECT_ROOT * "/test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
#incidents = CSV.File(PROJECT_ROOT * "/test/austin-data/austin_incidents.csv") |> DataFrame

210×44 Array{Bool,2}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1     1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1    

In [3]:
#regions - list of regions/neighborhoods where demands come from (1 ... 210)
#locations -list of stations (1 ... 44)
#demands - hours x regions = demands
regions = Int[parse(Int,string(x)) for x in names(hourly_calls[:,6:ncol(hourly_calls)])]
locations = collect(1:size(coverage,2))
adjacent = convert(Array, adjacent_nbhd[:,2:ncol(adjacent_nbhd)])[regions,regions] .> 0.5
demand = convert(Array,hourly_calls[:,6:end]);

In [4]:
#filter incidents
#incidents = incidents[.~ismissing.(incidents[!,:stn1_min]), :] # drop 44 calls that were "unreachable" (because all stations are reachable from each other)
#remove "faulty data" if events in the log do not come in order
# incidents[!,:interarrival_seconds] = [0; incidents[!,:arrival_seconds][2:end] - incidents[!,:arrival_seconds][1:end-1]]
# incidents[!,:isweekday] .= true
# incidents[!,:isweekday][incidents[!,:dow] .== "Sun"] .= false
# incidents[!,:isweekday][incidents[!,:dow] .== "Sat"] .= false;
# incidents = incidents[incidents[!,:isweekday],:]

#regions2index = Dict{Int,Int}(regions[i]=>i for i in 1:length(regions))

In [5]:
#= previously there was an error that didnt account for all the regions because it
 it only included regions for incidents. the regions without incidents are changed
 to 0=#
# for x in incidents[!,:neighborhood]
#     if(!haskey(regions2index,x))
#         regions2index[x] = 0
#     end
# end
# incidents[!,:neighborhood] = [regions2index[x] for x in incidents[!,:neighborhood]];

# calls = incidents[:,[[:hour,:dow,:month,:year,:neighborhood,:interarrival_seconds];
#                      Symbol[Symbol("stn$(i)_min") for i in locations]]]

# DataFrames.first(calls, 6)

In [6]:
# We focus on emergency calls during the "peak period" (8AM - 8PM),
# with the emergency calls from the first 3 month as our training set,
# and the subsequent emergency calls from the remaining months as our test set
peak_period = (hourly_calls[!,:hour] .>= 8) .* (hourly_calls[!,:hour] .<= 20)
indices = 1:DataFrames.nrow(hourly_calls);
train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
test_filter  = .~train_filter;
train_indices = indices[train_filter]
test_indices = indices[test_filter];

#will change the 30 later
p = DeploymentProblem(30, length(locations), length(regions), demand, train_indices,
                      test_indices, coverage[regions,:], Array{Bool,2}(adjacent));

In [7]:
amb_deployment = Dict{Symbol, Dict{Int, Vector{Int}}}()  

Dict{Symbol,Dict{Int64,Array{Int64,1}}}()

In [8]:
(deployment_model, name) = (next_dp -> StochasticDeployment(next_dp, nperiods=500), :Stochastic)
println("$name: ")
amb_deployment[name] = Dict{Int, Vector{Int}}()
namb = 40
println("$namb ")
p.nambulances = namb
println("time for model generation - stochastic $namb ambulances")
## make sure to include @time begin
@time model = deployment_model(p)
set_optimizer(model.m, Gurobi.Optimizer)
println("time for model solution - stochastic $namb ambulances")
@time AmbulanceDeployment.optimize!(model, p)


Stochastic: 
40 
time for model generation - stochastic 40 ambulances
Academic license - for non-commercial use only
 18.889233 seconds (70.06 M allocations: 5.055 GiB, 22.15% gc time)
Academic license - for non-commercial use only
time for model solution - stochastic 40 ambulances
Academic license - for non-commercial use only
Optimize a model with 126754 rows, 4702544 columns and 5902358 nonzeros
Variable types: 0 continuous, 4702544 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-06, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+01]
Found heuristic solution: objective 6644.0000000
Presolve removed 110313 rows and 4662566 columns
Presolve time: 4.17s
Presolved: 16441 rows, 39978 columns, 89622 nonzeros
Found heuristic solution: objective 2111.0045330
Variable types: 0 continuous, 39978 integer (24410 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    8383    6.0

In [9]:
amb_deployment[name][namb] = deployment(model)
println

println (generic function with 4 methods)

In [10]:
model.x

44-element Array{VariableRef,1}:
 x[1]
 x[2]
 x[3]
 x[4]
 x[5]
 x[6]
 x[7]
 x[8]
 x[9]
 x[10]
 x[11]
 x[12]
 x[13]
 ⋮
 x[33]
 x[34]
 x[35]
 x[36]
 x[37]
 x[38]
 x[39]
 x[40]
 x[41]
 x[42]
 x[43]
 x[44]

In [11]:
#save julia variable as .csv
deployment(model)
#or
x_save = Array{Int64}(undef, 44)
count = 1
for x in JuMP.value.(model.x)
    x_save[count] = convert(Int,x)
    count = count + 1
end

using DelimitedFiles
writedlm( "stoch_x.csv",  x_save, ',')
x_save

44-element Array{Int64,1}:
 2
 2
 1
 1
 1
 0
 1
 2
 2
 0
 0
 0
 0
 ⋮
 1
 2
 0
 0
 0
 0
 1
 0
 1
 0
 1
 1

In [12]:
#save julia variable as .csv
I = size(model.y)[1]
J = size(model.y)[2]
K = size(model.y)[3]
y_save = Array{Int64}(undef, I,J,K)
for i in 1:I
    for j in 1:J
        for k in 1:K
            y_save[i,j,k] = convert(Int,JuMP.value.(model.y[i,j,k]))
        end
    end
end



In [13]:
using DelimitedFiles
writedlm( "stoch_y.csv",  y_save, ',')

In [14]:
print(sum(JuMP.value.(model.y[:,:,100])))

6.0

In [15]:
d = convert(DataFrame,y_save[:,:,1])

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
sum(y_save[:,:,2])


31

In [17]:
open("stoch_y.json","w") do f
    write(f, json_string)
end

LoadError: [91mUndefVarError: json_string not defined[39m

In [18]:
Pkg.add("NPZ")

  Resolving package versions...
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Project.toml`
No Changes to `C:\Users\Owner\Documents\Austin\AmbulanceDeployment\AmbulanceDeployment.jl-legacy\Manifest.toml`


In [19]:
using NPZ
npzwrite("stoch_y.npz",y_save)